In [10]:
!nvidia-smi

Mon Apr 19 16:58:34 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   63C    P0    31W /  70W |    224MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [11]:
!pip install yfinance

In [12]:
import yfinance as yf
import numpy as np
import pandas as pd
import tensorflow as tf

In [13]:
data = yf.download("GOOGL", start="2018-01-01", interval='1d')

[*********************100%***********************]  1 of 1 completed


In [14]:
data.shape

(829, 6)

In [15]:
data.head()

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2018-01-02,1053.020020,1075.979980,1053.020020,1073.209961,1073.209961,1588300
2018-01-03,1073.930054,1096.099976,1073.430054,1091.520020,1091.520020,1565900
2018-01-04,1097.089966,1104.079956,1094.260010,1095.760010,1095.760010,1302600
2018-01-05,1103.449951,1113.579956,1101.800049,1110.290039,1110.290039,1512500
2018-01-08,1111.000000,1119.160034,1110.000000,1114.209961,1114.209961,1232200


In [16]:
from sklearn.preprocessing import MinMaxScaler
import pickle
from tqdm.notebook import tnrange

In [17]:
data = data[['Close','Volume']]
data.head(3)

,Close,Volume
Date,,
2018-01-02,1073.209961,1588300
2018-01-03,1091.520020,1565900
2018-01-04,1095.760010,1302600


In [18]:
test_lenght = data[(data.index>='2020-09-01')].shape[0]

In [19]:
def CreatFeature_and_Targets(data,feature_lenght):
  X = []
  Y = []

  for i in tnrange(len(data)-feature_lenght):
       X.append(data.iloc[i : i + feature_lenght,:].values)
       Y.append(data["Close"].values[i+feature_lenght])

       X = np.array(X)
       Y = np.array(Y)

  return X , Y

In [20]:
def CreateFeatures_and_Targets(data, feature_length):
    X = []
    Y = []

    for i in tnrange(len(data) - feature_length): 
        X.append(data.iloc[i : i + feature_length,:].values)
        Y.append(data["Close"].values[i+feature_length])

    X = np.array(X)
    Y = np.array(Y)

    return X , Y

In [21]:
X , Y = CreateFeatures_and_Targets(data , 32)

In [22]:
X.shape , Y.shape

((797, 32, 2), (797,))

In [23]:
X_train , X_test , Y_train,Y_test = X[:-test_lenght],X[:-test_lenght:],Y[:test_lenght],Y[-test_lenght:]

In [24]:
X_train.shape,Y_train.shape

((639, 32, 2), (158,))

In [25]:
class MultidimesionScaler():
  def _init_(self):
    self.scalers = []

    def fit_transform(self,X):
      total_dims = X.shape[2]
      for i in range(total_dims):
        Scalar = MultiScaler()
        X[:,:,i] = Scalar.fit_transform(X[:,:,1])
        self.scalars.append(Scaler)
        return X

        def transform(self,X):
          for i in range(X.shape[2]):
            X[:,:,1] = self.scalars[i].transform(X[:,:,i])
            return X

In [26]:
Feature_Scaler = MultidimesionScaler()
X_train = Feature_Scaler.fit_transform(X_train)
X_test = Feature_Scaler.transform(X_test)

AttributeError: ignored

In [27]:
Feature_Scaler = MultidimensionScaler()
X_train = Feature_Scaler.fit_transform(X_train)
X_test = Feature_Scaler.transform(X_test)

NameError: ignored

In [28]:
class MultiDimensionScaler():
    def __init__(self):
        self.scalers = []

    def fit_transform(self , X):
        total_dims = X.shape[2]
        for i in range(total_dims):
            Scaler = MinMaxScaler()
            X[:, :, i] = Scaler.fit_transform(X[:,:,i])
            self.scalers.append(Scaler)
        return X

    def transform(self , X):
        for i in range(X.shape[2]):
            X[:, :, i] = self.scalers[i].transform(X[:,:,i])
        return X 

In [29]:
Feature_Scaler = MultiDimensionScaler()
X_train = Feature_Scaler.fit_transform(X_train)
X_test = Feature_Scaler.transform(X_test)

In [30]:
Target_Scaler = MinMaxScaler()
Y_train = Target_Scaler.fit_transform(Y_train.reshape(-1,1))
Y_test = Target_Scaler.transform(Y_test.reshape(-1,1))

In [31]:
def save_object(obj , name : str):
    pickle_out = open(f"{name}.pck","wb")
    pickle.dump(obj, pickle_out)
    pickle_out.close()

def load_object(name : str):
    pickle_in = open(f"{name}.pck","rb")
    data = pickle.load(pickle_in)
    return data

In [32]:
save_object(Feature_Scaler , "Feature_Scaler")
save_object(Target_Scaler , "Target_Scaler")

In [33]:
data.sort_index(inplace=True)

In [34]:
data = data.loc[~data.index.duplicated(keep = 'first')]

In [35]:
data.tail(3)

,Close,Volume
Date,,
2021-04-15,2285.250000,1458700
2021-04-16,2282.750000,1313600
2021-04-19,2279.935059,713920


In [36]:
data.isnull().sum()

Close     0
Volume    0
dtype: int64

In [37]:
data.describe()

,Close,Volume
count,829.000000,8.290000e+02
mean,1330.899655,1.844087e+06
std,286.893813,8.562385e+05
min,984.669983,4.656000e+05
25%,1124.290039,1.302600e+06
50%,1217.770020,1.615700e+06
75%,1464.699951,2.111000e+06
max,2285.250000,6.658900e+06


In [38]:
import plotly.graph_objects as go

In [39]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index,y=data['Close'],mode = 'lines'))
fig.update_layout(height = 500,width = 900,xaxis_title = 'Date',yaxis_title = 'Close')
fig.show()

In [40]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Close'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Close')
fig.show()

In [41]:
fig = go.Figure()

fig.add_trace(go.Scatter(x = data.index , y = data['Volume'] , mode = 'lines'))
fig.update_layout(height = 500 , width = 900, 
                  xaxis_title='Date' , yaxis_title='Volume')
fig.show()

In [42]:
fig = go.Figure()

fig.add_trace(go.Scatter(x=data.index,y=data['Volume'],mode = 'lines'))
fig.update_layout(height=500,width=900,xaxis_title='Date',yaxis_title='Volume')
fig.show()

In [44]:
from tensorflow.keras.callbacks import ModelCheckpoint , ReduceLROnPlateau

save_best = ModelCheckpoint("best_weights.h5", monitor='val_loss', save_best_only=True, save_weights_only=True)
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.25,patience=4, min_lr=0.00001,verbose = 1)

In [45]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense , Dropout , LSTM , Bidirectional

model = Sequential()

model.add(Bidirectional(LSTM(512 ,return_sequences=True , recurrent_dropout=0.1, input_shape=(32, 2))))
model.add(LSTM(256 ,recurrent_dropout=0.1))
model.add(Dropout(0.3))
model.add(Dense(64 , activation='elu'))
model.add(Dropout(0.3))
model.add(Dense(32 , activation='elu'))
model.add(Dense(1 , activation='linear'))

In [48]:
optimizer = tf.keras.optimizers.SGD(learning_rate = 0.002)
model.compile(loss='mse', optimizer=optimizer)

In [49]:
history = model.fit(X_train, Y_train,
            epochs=10,
            batch_size = 1,
            verbose=1,
            shuffle=False ,
            validation_data=(X_test , Y_test),
            callbacks=[reduce_lr , save_best])

ValueError: ignored

In [50]:
model.summary()

ValueError: ignored

In [51]:
model.load_weights("best_weights.h5")

ValueError: ignored